In [2]:
import utils

import re
from collections import Counter

## Day 2: Red-Nosed Reports

[#](https://adventofcode.com/2024/day/2) We have a report per line, and need to figure out which reports are safe. 

A report is safe all numbers in it are increasing or decreasing by min 1 to max 3.

In [12]:
sample_input: str = """7 6 4 2 1
1 2 7 8 9
9 7 6 2 1
1 3 2 4 5
8 6 4 4 1
1 3 6 7 9
"""

puzzle_input = utils.get_input(2, splitlines=False)

In [11]:
def parse_input(input_str=sample_input, debug: bool = False):
    data = []
    for i, line in enumerate(input_str.splitlines()):
        numbers = [int(x) for x in re.findall(r"\d+", line)]
        if debug:
            print(f"Line {i}: '{line}' | Extracted: {numbers}")
        data.append(numbers)
    return data


data = parse_input(sample_input, True)

Line 0: '7 6 4 2 1' | Extracted: [7, 6, 4, 2, 1]
Line 1: '1 2 7 8 9' | Extracted: [1, 2, 7, 8, 9]
Line 2: '9 7 6 2 1' | Extracted: [9, 7, 6, 2, 1]
Line 3: '1 3 2 4 5' | Extracted: [1, 3, 2, 4, 5]
Line 4: '8 6 4 4 1' | Extracted: [8, 6, 4, 4, 1]
Line 5: '1 3 6 7 9' | Extracted: [1, 3, 6, 7, 9]


This is a bit ugly, but it works! There isn't much here... there is a builtin [itertools.pairwise](https://docs.python.org/3/library/itertools.html#itertools.pairwise) which was new to me, and would have done the job here, but to be safe I manually grabbed the number pairs.

In [91]:
def check_report(nums: list[int]):
    """returns True for valid reports, False for invalid"""

    # check list is sorted
    if not (nums == sorted(nums) or nums == sorted(nums, reverse=True)):
        return False

    for i in range(len(nums) - 1):
        x, y = nums[i : i + 2]

        diff = x - y

        match diff:
            case _ if diff < 0:
                if not (-3 <= x - y <= -1):
                    return False
            case _ if diff == 0:
                return False
            case _ if diff > 0:
                if not (1 <= x - y <= 3):
                    return False

    return True


check_report(data[0]), check_report(data[1])

(True, False)

In [70]:
def solve(inp: str = sample_input, debug: bool = False):
    data = parse_input(inp)

    ans = sum([check_report(report) for report in data])

    return {"result": ans}


assert solve(sample_input)["result"] == 2  # sample ans check

results = solve(puzzle_input, debug=False)
print(f"Part 1: {results["result"]}")

Part 1: 421


## Part 2

We have a additional safety system which can render resports with only one bad level safe.


In [82]:
def remove_bad_level(report, i, debug: bool = False):

    nums = report[:i] + report[i + 1 :]
    if debug:
        print(f"{report=}, {i=}, {nums=}")
    return nums


remove_bad_level(data[3], 1, True)

report=[1, 3, 2, 4, 5], i=1, nums=[1, 2, 4, 5]


[1, 2, 4, 5]

In [92]:
def check_report_2(nums: list[int], debug: bool = False):
    """returns True for valid reports, False for invalid"""
    if debug:
        print(f"{nums=}")

    if check_report(nums):
        return True

    for i in range(len(nums)):
        nums_2 = remove_bad_level(nums, i, debug=debug)

        if debug:
            print(f"Trying with level removed at index {i}: {nums_2}")

        if check_report(nums_2):
            return True
    return False


check_report_2(data[0]), check_report_2(data[3], True)

nums=[1, 3, 2, 4, 5]
report=[1, 3, 2, 4, 5], i=0, nums=[3, 2, 4, 5]
Trying with level removed at index 0: [3, 2, 4, 5]
report=[1, 3, 2, 4, 5], i=1, nums=[1, 2, 4, 5]
Trying with level removed at index 1: [1, 2, 4, 5]


(True, True)

In [93]:
def solve_2(inp: str = sample_input, debug: bool = False):
    data = parse_input(inp)
    ans = 0
    for report in data:
        if check_report_2(report):
            ans += 1
            if debug:
                print(f"{report=}")

    if debug:
        print(f"{ans=}")

    return {"result": ans}


solve_2(sample_input, debug=True)
assert solve_2(sample_input)["result"] == 4  # p2 sample input answer

results = solve_2(puzzle_input, debug=False)
print(f"\nPart 2: {results["result"]}")

report=[7, 6, 4, 2, 1]
report=[1, 3, 2, 4, 5]
report=[8, 6, 4, 4, 1]
report=[1, 3, 6, 7, 9]
ans=4

Part 2: 476
